In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_exercises').getOrCreate()

In [0]:
df = spark.read.table('college_csv')
df.show(2)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--

In [0]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [0]:
df.head(1)

Out[5]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [0]:
from pyspark.ml.feature import VectorAssembler
df.columns

Out[7]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
assembler = VectorAssembler(
    inputCols = [
        'Apps','Accept','Enroll','Top10perc',
        'Top25perc','F_Undergrad','P_Undergrad',
        'Outstate','Room_Board','Books','Personal',
        'PhD','Terminal','S_F_Ratio','perc_alumni',
        'Expend','Grad_Rate'
    ], 
    outputCol = 'features'
)

In [0]:
output = assembler.transform(df)

In [0]:
#Spark doesn't like strings in response variable --> let's convert to 0/1
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'Private', outputCol = 'PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [0]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [0]:
final_data = output_fixed.select('features','PrivateIndex')

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
#Let's do some classification models now that we have the data ready to go and prepped
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [0]:
from pyspark.ml import Pipeline

In [0]:
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex',featuresCol = 'features')
rfc = RandomForestClassifier(labelCol = 'PrivateIndex',featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'PrivateIndex',featuresCol = 'features')

In [0]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [0]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex')

print('DTC Results', my_binary_eval.evaluate(dtc_preds))
print('RFC Results', my_binary_eval.evaluate(rfc_preds))
print('GBT Results', my_binary_eval.evaluate(gbt_preds))

DTC Results 0.800735930735931
RFC Results 0.9908225108225108
GBT Results 0.9794372294372294
